## 安裝套件

In [ ]:
! pip install requests

## 使用Get抓取資料

In [ ]:
import requests
res = requests.get('http://www.tse.com.tw/ch/trading/fund/T86/T86.php')
res.text

## 查詢指令

In [ ]:
import requests
help(requests)
dir(requests)
help(requests.get)
?request.post

## 使用Post抓取資料

In [ ]:
import requests
url = 'http://www.twse.com.tw/ch/trading/fund/T86/T86.php'
payload = {
    'qdate':'106/04/13',
    'select2':'24',
    'sorting':'by_issue'
}
res = requests.post(url, data = payload)
res.text

## 使用BeautifulSoup4

In [ ]:
! pip install BeautifulSoup4

In [ ]:
from bs4 import BeautifulSoup
html_sample = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Document</title>
</head>
<body>
    <h1 id="title">Hello World</h1>
    <a href="#" class="link">This is lisk1</a>
    <a href="#link2" class="link">This is link2</a>
</body>
</html>
"""
soup = BeautifulSoup(html_sample, 'html.parser')

In [ ]:
soup.text

In [ ]:
a_list = soup.select('a') # 列表
a_list[0].text # 內容

## 取出ID or Class

In [ ]:
soup.select('#title')[0].text

In [ ]:
for link in soup.select('.link'):
    #print(link)
    #print(link.text)
    print(link['href'])

## Yahoo字典抓取句子範例

In [ ]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://tw.dictionary.yahoo.com/dictionary?p=apple')
res.text
soup = BeautifulSoup(res.text, 'html.parser')
# soup.text
sample = soup.select('#example')
print(sample[0].text)

## 抓取買賣交易表格資料

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'http://www.twse.com.tw/ch/trading/fund/T86/T86.php'
payload = {
    'qdate':'106/04/13',
    'select2':'24',
    'sorting':'by_issue'
}
res = requests.post(url, data = payload)
soup = BeautifulSoup(res.text, 'html.parser')
tb = soup.select('#tbl-sortable')

In [ ]:
for th in tb[0].select('th'):
    print(th.text)

## 使用Pandas

In [ ]:
# 安裝
! pip install pandas
! pip install html5lib

## 從網頁資料到Pandas表格資料

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas
url = 'http://www.twse.com.tw/ch/trading/fund/T86/T86.php'
payload = {
    'qdate':'106/04/13',
    'select2':'24',
    'sorting':'by_issue'
}
res = requests.post(url, data = payload)
soup = BeautifulSoup(res.text, 'html.parser')
tb = soup.select('#tbl-sortable')
table = tb[0].prettify('utf-8') # tab資料變成文字
dfs = pandas.read_html(table, encoding='utf-8')
stockdf = dfs[0]

In [ ]:
stockdf.columns

In [ ]:
stockdf['\n    證券\n    \n     代號\n    \n   ']==2330 #
stockdf[stockdf['\n    證券\n    \n     代號\n    \n   ']==2330] # 抓出指定證卷代號

In [ ]:
stockdf['\n    外資\n    \n     買賣超股數\n    \n   '].max()
stockdf[stockdf['\n    外資\n    \n     買賣超股數\n    \n   '] == stockdf['\n    外資\n    \n     買賣超股數\n    \n   '].max()]

In [ ]:
stockdf.sort_values( by= '\n    外資\n    \n     買賣超股數\n    \n   ', ascending=False).head()

In [ ]:
stockdf.to_excel('stock2.xlsx') # 儲存成excel

## Pandas 讀各各種檔案從剪貼簿

In [ ]:
import pandas
pd2 = pandas.read_clipboard()
pd2[0]

## Pandas-datareader 插件，直接讀取外部資料

In [ ]:
! pip install pandas-datareader

In [ ]:
from pandas_datareader import data
import datetime
tw2330 = data.get_data_yahoo('2330.TW', start=datetime.datetime(2014,1,1), end=datetime.datetime(2014,4,1) )
tw2330

## 時間與字串之間的轉換

In [ ]:
from datetime import datetime
# 時間轉字串
currenttime = datetime.now()
currenttime.strftime('%Y-%m-%d')

In [ ]:
# 字串轉時間
a = '2017-04-16 14:00'
datetime.strptime(a, '%Y-%m-%d %H:%M')

In [ ]:
# 往前幾天
from datetime import datetime, timedelta
currenttime = datetime.now()
currenttime - timedelta(days=1)

## 函式西元年 轉成 民國年

In [75]:
# 
from datetime import datetime, timedelta
def getTWDate(dt):
    year = int(dt.strftime('%Y') ) - 1911
    return "{0}/{1}".format(year, dt.strftime('%m/%d'))

getTWDate(datetime.now())
# 往前推10天日期
for i in range(1,11):
    curtime = datetime.now() - timedelta(days=i)
    print(getTWDate(curtime))

106/04/15
106/04/14
106/04/13
106/04/12
106/04/11
106/04/10
106/04/09
106/04/08
106/04/07
106/04/06


In [76]:
import requests
from bs4 import BeautifulSoup
import pandas
from datetime import datetime, timedelta

def getTWDate(dt):
    year = int(dt.strftime('%Y') ) - 1911
    return "{0}/{1}".format(year, dt.strftime('%m/%d'))

url = 'http://www.twse.com.tw/ch/trading/fund/T86/T86.php'

payload = {
'qdate':'106/04/13',
'select2':'24',
'sorting':'by_issue'
}

def getTradingVolume(dt):
    payload['qdate'] = getTWDate(dt)
    res  = requests.post(url, data = payload)
    soup = BeautifulSoup(res.text, 'html.parser')
    tbl  = soup.select('#tbl-sortable')[0]
    dfs  = pandas.read_html(tbl.prettify('utf-8'))
    stockdf  = dfs[0]
    stockdf['ymd'] = dt
    return stockdf


In [77]:
dfs = []
for i in range(1,10):
    dt = currenttime - timedelta(days = i)
    dfs.append(getTradingVolume(dt))

In [78]:
len(dfs)

9

In [79]:
import pandas
stockdf = pandas.concat(dfs, ignore_index = True)
stockdf.head()

,證券 代號,證券 名稱,外資 買進股數,外資 賣出股數,外資 買賣超股數,投信 買進股數,投信 賣出股數,投信 買賣超股數,自營商 買賣超股數,自營商 買進股數 (自行買賣),自營商 賣出股數 (自行買賣),自營商 買賣超股數 (自行買賣),自營商 買進股數 (避險),自營商 賣出股數 (避險),自營商 買賣超股數 (避險),三大法人 買賣超股數,ymd
0,2337.0,旺宏,10474000.0,4552000.0,5922000.0,0.0,0.0,0.0,-572000.0,230000.0,802000.0,-572000.0,0.0,0.0,0.0,5350000.0,2017-04-14 15:00:38.089872
1,3006.0,晶豪科,2101000.0,1033000.0,1068000.0,0.0,1000.0,-1000.0,566000.0,457000.0,436000.0,21000.0,837000.0,292000.0,545000.0,1633000.0,2017-04-14 15:00:38.089872
2,3661.0,世芯-KY,511000.0,341000.0,170000.0,450000.0,0.0,450000.0,-40000.0,0.0,40000.0,-40000.0,0.0,0.0,0.0,580000.0,2017-04-14 15:00:38.089872
3,3016.0,嘉晶,579000.0,32000.0,547000.0,0.0,1000.0,-1000.0,-60000.0,0.0,60000.0,-60000.0,0.0,0.0,0.0,486000.0,2017-04-14 15:00:38.089872
4,2436.0,偉詮電,489000.0,74000.0,415000.0,0.0,1000.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,414000.0,2017-04-14 15:00:38.089872


## 更改欄位名稱

In [91]:
new_columns = []
for ele in stockdf.columns:
#     print(ele.strip().replace(' ','').replace('\n',''))
    new_columns.append(ele.strip().replace(' ','').replace('\n',''))
#     print(''.join(ele.strip().split()))
print(new_columns)
stockdf.columns = new_columns

['證券代號', '證券名稱', '外資買進股數', '外資賣出股數', '外資買賣超股數', '投信買進股數', '投信賣出股數', '投信買賣超股數', '自營商買賣超股數', '自營商買進股數(自行買賣)', '自營商賣出股數(自行買賣)', '自營商買賣超股數(自行買賣)', '自營商買進股數(避險)', '自營商賣出股數(避險)', '自營商買賣超股數(避險)', '三大法人買賣超股數', 'ymd']


In [94]:
stockdf

,證券代號,證券名稱,外資買進股數,外資賣出股數,外資買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數,ymd
0,2337.0,旺宏,10474000.0,4552000.0,5922000.0,0.0,0.0,0.0,-572000.0,230000.0,802000.0,-572000.0,0.0,0.0,0.0,5350000.0,2017-04-14 15:00:38.089872
1,3006.0,晶豪科,2101000.0,1033000.0,1068000.0,0.0,1000.0,-1000.0,566000.0,457000.0,436000.0,21000.0,837000.0,292000.0,545000.0,1633000.0,2017-04-14 15:00:38.089872
2,3661.0,世芯-KY,511000.0,341000.0,170000.0,450000.0,0.0,450000.0,-40000.0,0.0,40000.0,-40000.0,0.0,0.0,0.0,580000.0,2017-04-14 15:00:38.089872
3,3016.0,嘉晶,579000.0,32000.0,547000.0,0.0,1000.0,-1000.0,-60000.0,0.0,60000.0,-60000.0,0.0,0.0,0.0,486000.0,2017-04-14 15:00:38.089872
4,2436.0,偉詮電,489000.0,74000.0,415000.0,0.0,1000.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,414000.0,2017-04-14 15:00:38.089872
5,3532.0,台勝科,557000.0,12000.0,545000.0,14000.0,88000.0,-74000.0,-67000.0,12000.0,79000.0,-67000.0,0.0,0.0,0.0,404000.0,2017-04-14 15:00:38.089872
6,2454.0,聯發科,1434000.0,1073283.0,360717.0,0.0,3000.0,-3000.0,-27000.0,0.0,22000.0,-22000.0,62000.0,67000.0,-5000.0,330717.0,2017-04-14 15:00:38.089872
7,4968.0,立積,103000.0,234000.0,-131000.0,293000.0,0.0,293000.0,118000.0,138000.0,20000.0,118000.0,0.0,0.0,0.0,280000.0,2017-04-14 15:00:38.089872
8,2325.0,矽品,467000.0,297000.0,170000.0,0.0,6000.0,-6000.0,78000.0,1000.0,0.0,1000.0,77000.0,0.0,77000.0,242000.0,2017-04-14 15:00:38.089872
9,3519.0,綠能,200000.0,14000.0,186000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186000.0,2017-04-14 15:00:38.089872


In [95]:
stockdf.to_excel('stock3.xlsx')

## 存入SQL Lite

In [97]:
import sqlite3
db = sqlite3.connect('finance.sqlite')
stockdf.to_sql(name='trading_volumn', con = db, if_exists='replace')
db.close

<function Connection.close>

## 從db讀入資料

In [100]:
import sqlite3
import pandas
with sqlite3.connect('finance.sqlite') as db:
    df2 = pandas.read_sql_query('select * from trading_volumn', con=db)
df2

,index,證券代號,證券名稱,外資買進股數,外資賣出股數,外資買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數,ymd
0,0,2337.0,旺宏,10474000.0,4552000.0,5922000.0,0.0,0.0,0.0,-572000.0,230000.0,802000.0,-572000.0,0.0,0.0,0.0,5350000.0,2017-04-14 15:00:38.089872
1,1,3006.0,晶豪科,2101000.0,1033000.0,1068000.0,0.0,1000.0,-1000.0,566000.0,457000.0,436000.0,21000.0,837000.0,292000.0,545000.0,1633000.0,2017-04-14 15:00:38.089872
2,2,3661.0,世芯-KY,511000.0,341000.0,170000.0,450000.0,0.0,450000.0,-40000.0,0.0,40000.0,-40000.0,0.0,0.0,0.0,580000.0,2017-04-14 15:00:38.089872
3,3,3016.0,嘉晶,579000.0,32000.0,547000.0,0.0,1000.0,-1000.0,-60000.0,0.0,60000.0,-60000.0,0.0,0.0,0.0,486000.0,2017-04-14 15:00:38.089872
4,4,2436.0,偉詮電,489000.0,74000.0,415000.0,0.0,1000.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,414000.0,2017-04-14 15:00:38.089872
5,5,3532.0,台勝科,557000.0,12000.0,545000.0,14000.0,88000.0,-74000.0,-67000.0,12000.0,79000.0,-67000.0,0.0,0.0,0.0,404000.0,2017-04-14 15:00:38.089872
6,6,2454.0,聯發科,1434000.0,1073283.0,360717.0,0.0,3000.0,-3000.0,-27000.0,0.0,22000.0,-22000.0,62000.0,67000.0,-5000.0,330717.0,2017-04-14 15:00:38.089872
7,7,4968.0,立積,103000.0,234000.0,-131000.0,293000.0,0.0,293000.0,118000.0,138000.0,20000.0,118000.0,0.0,0.0,0.0,280000.0,2017-04-14 15:00:38.089872
8,8,2325.0,矽品,467000.0,297000.0,170000.0,0.0,6000.0,-6000.0,78000.0,1000.0,0.0,1000.0,77000.0,0.0,77000.0,242000.0,2017-04-14 15:00:38.089872
9,9,3519.0,綠能,200000.0,14000.0,186000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186000.0,2017-04-14 15:00:38.089872


## 登入

In [112]:
import requests
from bs4 import BeautifulSoup
import pandas
from datetime import datetime, timedelta
payload = {
    'utf8':'✓',
    'authenticity_token':'vvvttPwQDJiTgRGT1m6SKILn5/b6aF5cxGPBsSnPzCMmnuJvYu0U+b3S8m4ERhI96KqfIT45o/9OaOVq3rggRQ==',
    'user[email]':'pellok2002@gmail.com',
    'user[password]':'abc123',
    'user[remember_me]':'1'
}
# 保存 登入 的Session
rs = requests.session()
# 取得csrf token
res = rs.get('https://statementdog.com/users/sign_in')
soup = BeautifulSoup(res.text, 'html.parser')
csrf_token = soup.select('meta[name=csrf-token]')
payload['authenticity_token'] = csrf_token[0]['content']
# 登入
res = rs.post('https://statementdog.com/users/sign_in', data=payload)
res.text


'<!DOCTYPE HTML PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\n<head>\n\t<script type="text/javascript">\n(function(i,s,o,g,r,a,m){i[\'GoogleAnalyticsObject\']=r;i[r]=i[r]||function(){\n(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\nm=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,\'script\',\'https://www.google-analytics.com/analytics.js\',\'ga\');\n\nga(\'create\', \'UA-17581981-1\', \'auto\');\nga(\'require\', \'displayfeatures\');\nga(\'require\', \'linkid\', \'linkid.js\');\nga(\'send\', \'pageview\');\n</script>\n\n\t\n\t\t<link rel="stylesheet" media="screen" href="/assets/feeds-ce1d8fdb306392ff1af588224fedc3b9.css" />\n\n\t<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="Dfa2DryuCn7853fDRQhNFS9zPmlDnFRgjdGdxvq2TU4KKGJoSGeq1HxJnAa5Nu9HVOb5rxAQyBDRN1tlxtcj3Q==" /

## 抓取檔案

In [ ]:
import requests
with open('stock.jpg') as f:
    res = requests.get('url')
    f.write(res.content)

## 抓取JS ajas 資料